In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Load data

In[ ]:

In[28]:

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

rom aif360.datasets import BinaryLabelDataset<br>
rom aif360.metrics import BinaryLabelDatasetMetric

Citizens:

In[29]:

In [ ]:
pathRoot="../Data/"

In[30]:

In [ ]:
pathCitizens=pathRoot+"borgere_hmi_Rasmus_BorgerId_Gender_BirthYear.xlsx"
citizensData=pd.read_excel(pathCitizens,sheet_name=5)

In[31]:

In [ ]:
citizensData["INDLEVERINGSDATO"]=pd.to_datetime(citizensData["INDLEVERINGSDATO"], errors='coerce')

Patientdata

In[32]:

In [ ]:
pathPatientData=pathRoot+"DrPatientData_RasmusPlusBorgerIdMinusCPR.xlsx"
patientData=pd.read_excel(pathPatientData)

Observations

In[33]:

In [ ]:
pathObservationsData=pathRoot+"Observationer_Rasmus_BorgerIdOgGenderMinusCPR.xlsx"
observationsData=pd.read_excel(pathObservationsData)

Training:

In[34]:

In [ ]:
pathTrainingData=pathRoot+"TrÃ¦ning_Rasmus_PlusBorgerIdOgGenderMinusCPR.xlsx"
trainingData=pd.read_excel(pathTrainingData)

# Citizens

In[8]:

In [ ]:
citizensData.dtypes

In[9]:

In [ ]:
citizensData.head(1)

In[10]:

In [ ]:
citizensData.columns

In[11]:

In [ ]:
citizensData["HJAELPEMIDDELHMINAVN"].nunique()

The fraction of male females:

In[12]:

In [ ]:
citizensData[["Gender","BorgerID"]].groupby(["Gender"]).nunique()/citizensData["BorgerID"].nunique()

In[13]:

In [ ]:
sns.barplot(data=citizensData[["Gender","BorgerID"]].astype({'BorgerID': 'int32'}),x="Gender",y="BorgerID")

# Patient data

In[14]:

In [ ]:
patientData.dtypes

In[15]:

In [ ]:
patientData["Screening Content"][0].split("#")[0].split(";")

In[16]:

In [ ]:
patientData["Screening Content"][0].split("#")[1].split(";")

In[17]:

In [ ]:
patientData["Screening Content"][1].split("#")[0].split(";")

In[18]:

In [ ]:
for i in range(len(patientData["Screening Content"][0].split("#"))):
    for j in range(len(patientData["Screening Content"][0].split("#")[i].split(";"))):
        print(patientData["Screening Content"][0].split("#")[i].split(";")[j])

### Renaming BorgerId

In[48]:

In [ ]:
citizensData.head(5)

In[49]:

In [ ]:
citizensData=citizensData.rename(columns={"BorgerID":"BorgerId"})
citizensData.head(5)

# Join the data

Hvordan bÃ¸r det gÃ¸res?<br>
<br>
<br>
select *, case exists(falddata) then 1 else 0 end as "Faldet" <br><br>
from hjÃ¦lpemiddel<br><br>
     left join generelt data <br><br>
     left join falddata <br><br>


In[50]:

In [ ]:
observationsData

In[51]:

In [ ]:
citizensData

In[54]:

ttps://stackoverflow.com/questions/37292872/how-can-i-one-hot-encode-in-python

In [ ]:
one_hot = pd.get_dummies(citizensData['HJAELPEMIDDELHMINAVN'])
# Drop column B as it is now encoded
citizensData_updated = citizensData[["BorgerId","Gender","Birth Year"]]
# Join the encoded df
citizensData_updated = citizensData_updated.join(one_hot)
citizensData_updated

In[55]:

In [ ]:
citizensData_updated=citizensData_updated.groupby('BorgerId').max()

In[66]:

In [ ]:
citizensData_updated.shape

In[67]:

In [ ]:
citizensData_updated.dtypes

In[57]:

In [ ]:
citizensData_updated = citizensData_updated.reset_index()

In[58]:

In [ ]:
citizensData_updated.drop_duplicates(subset=['BorgerId'])

In[59]:

In [ ]:
citizensData_updated.dropna(subset=['BorgerId'])

In[ ]:

In[70]:

In [ ]:
merged_new=pd.merge(
    left=observationsData,
    right=citizensData_updated, 
    how="left",
    on="BorgerId",
    #left_on="BorgerId",
    #right_on="BorgerId",
    left_index=False,
    right_index=False,
    #sort=True,
    suffixes=("_obs", "_cit"),
   # copy=True,
    indicator=False,
   # validate=None,
)

In[71]:

In [ ]:
merged_new

In[63]:

In [ ]:
merged_new_clean = merged_new.dropna(subset=['BorgerId'])
merged_new_clean = merged_new_clean.dropna(subset=['Birth Year'])
merged_new_clean

In[65]:

In [ ]:
merged_new_clean.drop_duplicates(subset=['BorgerId'])

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[27]:

In [ ]:
merged=pd.merge(
    left=citizensData_updated,
    right=observationsData,
    how="left",
    on="BorgerId",
    #left_on="BorgerId",
    #right_on="BorgerId",
    left_index=False,
    right_index=False,
    #sort=True,
    suffixes=("_cit", "_obs"),
   # copy=True,
    indicator=False,
   # validate=None,
)

In[28]:

In [ ]:
merged["Fall"]=merged["Dato"].isnull().apply(lambda x: 1 if x is False else 0)

erged=merged[merged["Svar lang"].notna()]

In[29]:

In [ ]:
merged.head(5)

In[30]:

In [ ]:
merged=merged.drop(['Gender_obs','Dato','SpÃ¸rgmÃ¥l lang','Svar lang'], axis=1)

In[31]:

In [ ]:
merged["Gender"]=merged["Gender_cit"].apply(lambda x: 0 if x=="FEMALE" else 1)
merged=merged.drop(["Gender_cit"],axis=1)

In[32]:

In [ ]:
merged=merged.drop_duplicates()

In[33]:

In [ ]:
merged.BorgerId.value_counts()

In[34]:

In [ ]:
merged.Fall.mean()

## Sanity check

In[35]:

In [ ]:
observationsData.head(5)

In[38]:

In [ ]:
citizensData_updated=citizensData_updated.reset_index()

In[39]:

In [ ]:
citizensData_updated.head(5)

In[40]:

In [ ]:
observationsData.BorgerId.value_counts()

In[41]:

In [ ]:
citizensData_updated.BorgerId.value_counts()

In[42]:

In [ ]:
merged.BorgerId.value_counts()

In[43]:

In [ ]:
merged.Fall.mean()

In[44]:

In [ ]:
merged.head(5)

In[46]:

In [ ]:
merged.to_csv("../Data/dummy_data.csv")

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

## Support vector machine

In[127]:

In [ ]:
X_cols=[name for name in merged.columns.tolist() if name not in ["Fall","BorgerId"]]

In[128]:

In [ ]:
X=merged[X_cols]

In[129]:

In [ ]:
y=merged["Fall"]

In[130]:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

In[131]:

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

In[132]:

In [ ]:
clf.score(X_test, y_test)

In[133]:

In [ ]:
preds=clf.predict(X_test)

In[135]:

In [ ]:
preds

## Equalized odds

In[136]:

In [ ]:
X_cols=[name for name in merged.columns.tolist() if name not in ["Fall","BorgerId"]]

In[137]:

In [ ]:
train, test =     train_test_split(merged.drop(["BorgerId"],axis=1), test_size = 0.2, random_state = 123)

In[138]:

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In[139]:

In [ ]:
clf = svm.SVC()
clf.fit(train[X_cols], train["Fall"])

In[140]:

In [ ]:
clf.score(test[X_cols], test["Fall"])

In[141]:

In [ ]:
y_pred_f1=clf.predict(test[X_cols])

In[142]:

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(test["Fall"], y_pred_f1)

In[143]:

In [ ]:
y_pred_f1[y_pred_f1==0]

Making predictions:

In[144]:

In [ ]:
train["output"]=clf.predict(train[X_cols])

In[145]:

In [ ]:
test["output"]=clf.predict(test[X_cols])

In[146]:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(test["Fall"].ravel(), test["output"].ravel())
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1])
disp.plot() 

In[ ]:

In[ ]:

# Equalized odds

Rename trainingset TRUE falls to be named "selected_col". <br><br>
Rename trainingset PREDICTED falls to be named "selected_col".

In[147]:

In [ ]:
trainset_renamed=train.rename(columns={"Fall": "selected_col"}).drop(['output'], axis=1)
predicted_trainset_renamed=train.rename(columns={"output": "selected_col"}).drop(['Fall'], axis=1)

Rename testset TRUE falls to be named "selected_col". <br><br>
Rename testset PREDICTED falls to be named "selected_col".

In[148]:

In [ ]:
testset_renamed=test.rename(columns={"Fall": "selected_col"}).drop(['output'], axis=1)
predicted_testset_renamed=test.rename(columns={"output": "selected_col"}).drop(['Fall'], axis=1)

Needs all columns when using BinaryLabelDataset-class:

In[149]:

In [ ]:
all_cols=X_cols
all_cols=all_cols.append("selected_col")

Create training binary dataset:

In[171]:

rain TRUE

In [ ]:
train_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=trainset_renamed,
                                label_names=['selected_col'],
                                protected_attribute_names=['Gender'],
                                unprivileged_protected_attributes=['0'])
#TRAIN PREDICTED
pred_train_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=predicted_trainset_renamed,
                                label_names=['selected_col'], #label_names=['preds'],
                                protected_attribute_names=['Gender'],
                                unprivileged_protected_attributes=['0'])

Create test binary dataset:

In[172]:

est TRUE

In [ ]:
test_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=testset_renamed,
                                label_names=['selected_col'],
                                protected_attribute_names=['Gender'],
                                unprivileged_protected_attributes=['0'])
#test PREDICTED
pred_test_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=predicted_testset_renamed,
                                label_names=['selected_col'], #label_names=['preds'],
                                protected_attribute_names=['Gender'],
                                unprivileged_protected_attributes=['0'])

In[ ]:

In[194]:

ttps://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.postprocessing.EqOddsPostprocessing.html#aif360.algorithms.postprocessing.EqOddsPostprocessing

In [ ]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
new_eq=EqOddsPostprocessing(unprivileged_groups= [{'Gender': 0}],privileged_groups=[{'Gender': 1}])

In[195]:

In [ ]:
new_eq.fit(train_BLD,pred_train_BLD)

In[196]:

In [ ]:
prediction_test = new_eq.predict(pred_test_BLD)

In[197]:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(test["Fall"].ravel(), prediction_test.labels.ravel())
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1])
disp.plot() 

In[188]:

ttps://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.postprocessing.EqOddsPostprocessing.html#aif360.algorithms.postprocessing.EqOddsPostprocessing

In [ ]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
new_ca=CalibratedEqOddsPostprocessing(unprivileged_groups= [{'Gender': 0}],privileged_groups=[{'Gender': 1}])

In[189]:

In [ ]:
new_ca.fit(train_BLD,pred_train_BLD)

In[192]:

In [ ]:
prediction_test = new_ca.predict(pred_test_BLD)

In[193]:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(test["Fall"].ravel(), prediction_test.labels.ravel())
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1])
disp.plot() 

In[ ]:

In [ ]:
test["Fall"].ravel()==